# Pseudo dataset
* Get sample data [here](https://github.com/fuyu-quant/IBLM/tree/main/datasets).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/IBLM/blob/main/examples/iblmodel_pseudodata.ipynb)

In [1]:
%%capture
!pip install git+https://github.com/fuyu-quant/IBLM.git

In [2]:
import pkg_resources
print(pkg_resources.get_distribution('IBLM').version)

0.0.13


### Training

In [3]:
import pandas as pd
import numpy as np
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import IBLMClassifier


import os
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

In [4]:
df = pd.read_csv('/content/pseudodata_train.csv')
x_train = df.drop('E', axis=1)
y_train = df['E']

In [41]:
llm_model_name = 'gpt-4'

params = {
    'columns_name': True
    }

iblm = IBLMClassifier(llm_model_name=llm_model_name, params=params)

In [42]:
model = iblm.fit(x_train, y_train, model_name = 'pseudodata', file_path='/content/')

> Start of model creating.
Tokens Used: 7659
	Prompt Tokens: 7483
	Completion Tokens: 176
Successful Requests: 1
Total Cost (USD): $0.23504999999999995


In [43]:
# Code of the model created
print(model)

import numpy as np
def predict(x):
    df = x.copy()
    output = []
    for index, row in df.iterrows():

        # Feature creation and data preprocessing
        A, B, C, D = row['A'], row['B'], row['C'], row['D']

        # Logic for prediction
        if A > 0 and C < 0:
            y = 0
        elif A < 0 and C > 0:
            y = 1
        elif B > 0 and D < 0:
            y = 0
        elif B < 0 and D > 0:
            y = 1
        else:
            y = 0.5

        y = 1 / (1 + np.exp(-y))
        output.append(y)
    return np.array(output)


### Prediction

In [46]:
df = pd.read_csv('/content/pseudodata_test.csv')
x_test = df.drop('E', axis=1)
y_test = df['E']

In [47]:
y_proba = iblm.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

In [48]:
accuracy = round(accuracy_score(y_test, y_pred),4)
print(f'Accuracy: {accuracy}')

# Precision
precision = round(precision_score(y_test, y_pred),4)
print(f'Precision: {precision}')

# Recall
recall = round(recall_score(y_test, y_pred),4)
print(f'Recall: {recall}')

# F1 score
f1 = round(f1_score(y_test, y_pred),4)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = round(roc_auc_score(y_test, y_proba),4)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.827
Precision: 0.795
Recall: 0.8843
F1 score: 0.8373
ROC-AUC: 0.8654


### Prediction from external files


In [ ]:
import pseudodata

y_proba = pseudodata.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

In [20]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.415
Precision: 0.43103448275862066
Recall: 0.49504950495049505
F1 score: 0.4608294930875576
ROC-AUC: 0.35973597359735976


### Interpretation of results

In [57]:
description = iblm.interpret()

Tokens Used: 881
	Prompt Tokens: 537
	Completion Tokens: 344
Successful Requests: 1
Total Cost (USD): $0.036750000000000005


In [58]:
print(description)

- Data preprocessing:
    - Fill missing 'age' values with the median age.
    - Fill missing 'fare' values with the median fare.
    - Fill missing 'embarked' values with the mode (most frequent) of the 'embarked' column.

- Feature creation:
    - Create a new binary feature 'is_female' based on the 'sex' column.
    - Create a new binary feature 'is_child' based on the 'age' column.
    - Create a new binary feature 'is_adult_male' based on the 'adult_male' column.
    - Create a new binary feature 'is_alone' based on the 'alone' column.
    - Create new binary features 'is_first_class', 'is_second_class', and 'is_third_class' based on the 'pclass' column.
    - Create new binary features 'embarked_C', 'embarked_Q', and 'embarked_S' based on the 'embarked' column.

- Prediction logic:
    - Initialize a variable 'y' to 0.
    - Add or subtract weights to 'y' based on the created binary features.
    - Apply the logistic function (sigmoid) to 'y' to get the final prediction.
    - Ap